In [ ]:
'''
This script will take a folder full of video files and
extract stills from each video at a specified interval.
Naming convention for stills is video file name + underscore
+ MMSS (minutes and seconds)
'''

# Import libraries
import cv2
import os
from google.colab import drive

# Set parameters
# Folder paths based on Google Drive
input_folder_path = '/content/drive/MyDrive/FreeFuse_Project/Source_Videos/Alloy Personal Training'
output_folder_path = '/content/drive/MyDrive/FreeFuse_Project/Extracted_Stills'

# Time interval for capturing frames
capture_interval_seconds = 2

# Start time (in seconds) to skip at beginning of each video
start_time_seconds = 1

# Mount your Google Drive
print("Connecting to Google Drive...")
drive.mount('/content/drive')
print("Google Drive connected.")

# Create the output directory if it doesn't exist
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
    print(f"Created output folder: {output_folder_path}")

# Get a list of all video files in the input folder
try:
    all_files = os.listdir(input_folder_path)
    video_files = [f for f in all_files if f.lower().endswith(('.mp4', '.mov', '.avi'))]
except FileNotFoundError:
    print(f"ERROR: Input folder not found at '{input_folder_path}'")
    print("Please make sure the folder exists and the path is correct in the parameters section.")
    video_files = []

# Loop to iterate over each video file
if not video_files:
    print("No video files found in the specified input folder.")
else:
    print(f"Found {len(video_files)} video(s) to process in '{input_folder_path}'.")

    for i, video_filename in enumerate(video_files):
        print(f"\n--- Processing video {i+1} of {len(video_files)}: {video_filename} ---")

        full_video_path = os.path.join(input_folder_path, video_filename)
        video_capture = cv2.VideoCapture(full_video_path)

        if not video_capture.isOpened():
            print(f"Error: Could not open video file {video_filename}. Skipping.")
            continue

        fps = video_capture.get(cv2.CAP_PROP_FPS)

        if start_time_seconds > 0:
            start_frame = int(start_time_seconds * fps)
            video_capture.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
            print(f"Skipping to {start_time_seconds} second(s) into the video.")

        next_capture_time = start_time_seconds
        saved_frame_count = 0

        while video_capture.isOpened():
            success, frame = video_capture.read()
            if not success:
                break

            current_time_seconds = video_capture.get(cv2.CAP_PROP_POS_MSEC) / 1000.0

            if current_time_seconds >= next_capture_time:
                minutes = int(current_time_seconds / 60)
                seconds = int(current_time_seconds % 60)
                timestamp = f"{minutes:02d}{seconds:02d}"

                video_name_base = os.path.splitext(video_filename)[0]
                output_filename = f"{video_name_base}_{timestamp}.jpg"
                full_output_path = os.path.join(output_folder_path, output_filename)

                cv2.imwrite(full_output_path, frame)
                print(f"  Saved frame: {output_filename}")

                next_capture_time += capture_interval_seconds
                saved_frame_count += 1

        video_capture.release()
        print(f"Finished processing {video_filename}. Saved {saved_frame_count} frames.")

    print("\n--- All videos processed! ---")